In [ ]:
# !pip install scikit-learn==0.22.2

In [ ]:
# !pip install scanpy==1.5.1

In [ ]:
# !pip install jgraph

In [1]:
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


In [2]:
def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [3]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [4]:
category = "imbalanced_data"# "balanced_data"
files = glob2.glob(f'../R/simulated_data/{category}/*.h5')
files = [f[len(f"{path}R/simulated_data/{category}/"):-3] for f in files]
files

['data_1c8',
 'data_-1c4',
 'data_-1c8',
 'data_0c4',
 'data_0c8',
 'data_0c16',
 'data_1.5c4',
 'data_1c4',
 'data_1.5c8',
 'data_1.5c16',
 'data_-1c16',
 'data_1c16']

In [10]:
df = pd.DataFrame(columns = ["dataset", "sczi", "run"])
for dataset in files:
    print(f">>> Dataset {dataset} ")
    data_mat = h5py.File(f"{path}R/simulated_data/{category}/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    print(f"Dropout level {data_mat['dropout'][0]}")
    dp = data_mat['dropout'][0]
    print(np.where(X ==0)[0].shape[0]/(X.shape[0]*X.shape[1]))
    X = np.ceil(X).astype(np.int)
    count_X = X
    print(X.shape, count_X.shape)
    orig_X = X.copy()
    adata = sc.AnnData(X)
    adata.obs['Group'] = Y

    X = np.ceil(X).astype(np.int)
    count_X = X
    print(X.shape, count_X.shape)

    adata = sc.AnnData(X)
    adata.obs['Group'] = Y
    adata = normalize(adata,
                      copy=True,
                      highly_genes=args.highly_genes,
                      size_factors=True,
                      normalize_input=True,
                      logtrans_input=True)
    X = adata.X.astype(np.float32)
    Y = np.array(adata.obs["Group"])
    print(X.shape, count_X.shape)

    high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
    count_X = count_X[:, high_variable]
    size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                           1).astype(np.float32)
    cluster_number = int(max(Y) - min(Y) + 1)
    args.dims[0]=X.shape[1]
    
    for run in range(3):
        seed = run
        np.random.seed(seed)
        tf.reset_default_graph()
        chencluster = autoencoder(args.dataname, args.distribution,
                                  args.self_training, args.dims, cluster_number,
                                  args.t_alpha, args.alpha, args.gamma,
                                  args.learning_rate, args.noise_sd)
        chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                             args.pretrain_epoch, args.gpu_option)

        chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                              args.funetrain_epoch, args.update_epoch, args.error, Y)

        kmeans_accuracy = np.around(cluster_acc(Y, chencluster.kmeans_pred), 5)
        kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
        kmeans_NMI = np.around(
            normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)
        accuracy = np.around(cluster_acc(Y, chencluster.Y_pred), 5)
        ARI = np.around(adjusted_rand_score(Y, chencluster.Y_pred), 5)
        NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)

        df.loc[df.shape[0]] = [dataset, ARI, run]

        df.to_pickle(f"{path}output/pickle_results/{category}/{category}_sczi.pkl")

>>> Dataset data_0c32 
Dropout level 0.16529533856152173
0.35246906666666666
(3000, 2500) (3000, 2500)
(3000, 2500) (3000, 2500)
(3000, 499) (3000, 2500)
begin the pretraining
begin the funetraining
ARI 0.00691, NMI 0.07425
0 ARI 0.00691, NMI 0.07425


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.00582, NMI 0.07133
0 ARI 0.00579, NMI 0.07124
10 ARI 0.00576, NMI 0.07119
20 ARI 0.00577, NMI 0.07054
30 ARI 0.00592, NMI 0.07118
40 ARI 0.00579, NMI 0.07058
50 ARI 0.00565, NMI 0.07003
60 ARI 0.0058, NMI 0.07054
70 ARI 0.00577, NMI 0.07042
80 ARI 0.00574, NMI 0.07001
90 ARI 0.00577, NMI 0.07038
100 ARI 0.00576, NMI 0.07028
110 ARI 0.00588, NMI 0.07109
120 ARI 0.00584, NMI 0.0709
130 ARI 0.0059, NMI 0.07126
140 ARI 0.00594, NMI 0.07148
150 ARI 0.00589, NMI 0.07165
160 ARI 0.00589, NMI 0.07147
170 ARI 0.00588, NMI 0.07146
180 ARI 0.00597, NMI 0.07189
190 ARI 0.00595, NMI 0.07166
200 ARI 0.00593, NMI 0.07153
210 ARI 0.00592, NMI 0.07159
220 ARI 0.00596, NMI 0.07193
230 ARI 0.00585, NMI 0.07151
240 ARI 0.00587, NMI 0.07153
250 ARI 0.00591, NMI 0.07166
260 ARI 0.00593, NMI 0.07172
270 ARI 0.00604, NMI 0.07203
280 ARI 0.00607, NMI 0.07231
290 ARI 0.00611, NMI 0.07257
300 ARI 0.00607, NMI 0.07242
310 ARI 0.00612, NMI 0.07263
320 ARI 0.00615,

/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.00832, NMI 0.08022
0 ARI 0.00836, NMI 0.08071
10 ARI 0.00855, NMI 0.08083
20 ARI 0.00884, NMI 0.08226
30 ARI 0.0087, NMI 0.08335
40 ARI 0.00847, NMI 0.08298
50 ARI 0.00831, NMI 0.08234
60 ARI 0.00839, NMI 0.0827
70 ARI 0.00838, NMI 0.08252
80 ARI 0.00852, NMI 0.08319
90 ARI 0.00849, NMI 0.08308
100 ARI 0.00847, NMI 0.08292
110 ARI 0.00848, NMI 0.08292
120 ARI 0.00838, NMI 0.08258
130 ARI 0.00841, NMI 0.08255
140 ARI 0.00838, NMI 0.08247
150 ARI 0.00839, NMI 0.08225
160 ARI 0.0084, NMI 0.0824
170 ARI 0.00845, NMI 0.08261
180 ARI 0.00846, NMI 0.08262
190 ARI 0.00841, NMI 0.08238
200 ARI 0.00839, NMI 0.08231
210 ARI 0.00845, NMI 0.0823
220 ARI 0.00842, NMI 0.08228
230 ARI 0.00841, NMI 0.08239
240 ARI 0.00836, NMI 0.08207
250 ARI 0.0084, NMI 0.08251
260 ARI 0.00843, NMI 0.08247
270 ARI 0.00842, NMI 0.08268
280 ARI 0.0084, NMI 0.08243
290 ARI 0.00845, NMI 0.0825
300 ARI 0.00844, NMI 0.08247
310 ARI 0.00843, NMI 0.08265
320 ARI 0.00844, NMI 

...

/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.0154, NMI 0.09199
0 ARI 0.01542, NMI 0.09213
10 ARI 0.01552, NMI 0.09248
20 ARI 0.01589, NMI 0.09347
30 ARI 0.0164, NMI 0.09503
40 ARI 0.01647, NMI 0.09507
50 ARI 0.01654, NMI 0.09554
60 ARI 0.01654, NMI 0.09574
70 ARI 0.01685, NMI 0.09591
80 ARI 0.01719, NMI 0.09646
90 ARI 0.01752, NMI 0.09654
100 ARI 0.01757, NMI 0.09674
110 ARI 0.01757, NMI 0.0967
120 ARI 0.0176, NMI 0.09682
130 ARI 0.01768, NMI 0.09728
140 ARI 0.01792, NMI 0.09747
150 ARI 0.01793, NMI 0.09749
160 ARI 0.01793, NMI 0.0975
170 ARI 0.01794, NMI 0.09752
180 ARI 0.01786, NMI 0.09715
190 ARI 0.01795, NMI 0.09736
200 ARI 0.01785, NMI 0.09725
210 ARI 0.01798, NMI 0.09777
220 ARI 0.01796, NMI 0.0977
230 ARI 0.01795, NMI 0.09762
240 ARI 0.01796, NMI 0.0978
250 ARI 0.01796, NMI 0.09785
260 ARI 0.01787, NMI 0.09745
270 ARI 0.01791, NMI 0.09762
280 ARI 0.01794, NMI 0.09773
290 ARI 0.01792, NMI 0.0975
300 ARI 0.01793, NMI 0.09741
310 ARI 0.0179, NMI 0.09732
320 ARI 0.01788, NMI 0

/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


begin the pretraining
begin the funetraining
ARI 0.02137, NMI 0.1054
0 ARI 0.0214, NMI 0.10544
10 ARI 0.02131, NMI 0.10501
20 ARI 0.02131, NMI 0.10522
30 ARI 0.02126, NMI 0.10489
40 ARI 0.0212, NMI 0.10453
50 ARI 0.02111, NMI 0.10439
60 ARI 0.02105, NMI 0.10413
70 ARI 0.02125, NMI 0.10456
80 ARI 0.02128, NMI 0.10476
90 ARI 0.02113, NMI 0.10451
100 ARI 0.02101, NMI 0.10438
110 ARI 0.02112, NMI 0.10443
120 ARI 0.02126, NMI 0.10444
130 ARI 0.02088, NMI 0.10389
140 ARI 0.02124, NMI 0.10449
150 ARI 0.02132, NMI 0.10481
160 ARI 0.0214, NMI 0.10493
170 ARI 0.02144, NMI 0.10489
180 ARI 0.02148, NMI 0.10498
190 ARI 0.02146, NMI 0.10492


/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [11]:
df


,dataset,sczi,run
0,data_1c8,0.06997,0
1,data_1c8,0.41693,1
2,data_1c8,0.32892,2
3,data_-1c4,0.66109,0
4,data_-1c4,0.64426,1
5,data_-1c4,0.69202,2
6,data_-1c8,0.48873,0
7,data_-1c8,0.50074,1
8,data_-1c8,0.56503,2
9,data_0c4,0.63704,0


In [12]:
df.groupby("dataset").mean()

,sczi
dataset,
data_-1c16,0.232413
data_-1c32,0.019613
data_-1c4,0.665790
data_-1c8,0.518167
data_0c16,0.104137
data_0c32,0.008883
data_0c4,0.571307
data_0c8,0.435410
data_1.5c16,0.012817
